In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torchvision.ops
import torch.optim as optim
import torch.nn.functional as F

class DeformableConv2d(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size=3,
                 stride=1,
                 padding=1,
                 dilation=1,
                 bias=False):
        super(DeformableConv2d, self).__init__()

        assert type(kernel_size) == tuple or type(kernel_size) == int

        kernel_size = kernel_size if type(kernel_size) == tuple else (kernel_size, kernel_size)
        self.stride = stride if type(stride) == tuple else (stride, stride)
        self.padding = padding
        self.dilation = dilation

        self.offset_conv = nn.Conv2d(in_channels,
                                     2 * kernel_size[0] * kernel_size[1],
                                     kernel_size=kernel_size,
                                     stride=stride,
                                     padding=self.padding,
                                     dilation=self.dilation,
                                     bias=True)

        nn.init.constant_(self.offset_conv.weight, 0.)
        nn.init.constant_(self.offset_conv.bias, 0.)

        self.modulator_conv = nn.Conv2d(in_channels,
                                        1 * kernel_size[0] * kernel_size[1],
                                        kernel_size=kernel_size,
                                        stride=stride,
                                        padding=self.padding,
                                        dilation=self.dilation,
                                        bias=True)

        nn.init.constant_(self.modulator_conv.weight, 0.)
        nn.init.constant_(self.modulator_conv.bias, 0.)

        self.regular_conv = nn.Conv2d(in_channels=in_channels,
                                      out_channels=out_channels,
                                      kernel_size=kernel_size,
                                      stride=stride,
                                      padding=self.padding,
                                      dilation=self.dilation,
                                      bias=bias)

    def forward(self, x):
        offset = self.offset_conv(x)
        modulator = 2. * torch.sigmoid(self.modulator_conv(x))
        x = torchvision.ops.deform_conv2d(input=x,
                                          offset=offset,
                                          weight=self.regular_conv.weight,
                                          bias=self.regular_conv.bias,
                                          padding=self.padding,
                                          mask=modulator,
                                          stride=self.stride,
                                          dilation=self.dilation)
        return x

class Attention(nn.Module):
    def __init__(self, in_channels):
        super(Attention, self).__init__()
        self.attention = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // 8, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels // 8, in_channels, kernel_size=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        attention = self.attention(x)
        return x * attention


class CBAM(nn.Module):
    def __init__(self, channels, r):
        super(CBAM, self).__init__()
        self.channels = channels
        self.r = r
        self.sam = SAM(bias=False)
        self.cam = CAM(channels=self.channels, r=self.r)

    def forward(self, x):
        output = self.cam(x)
        output = self.sam(output)
        return output + x


class CAM(nn.Module):
    def __init__(self, channels, r):
        super(CAM, self).__init__()
        self.channels = channels
        self.r = r
        self.linear = nn.Sequential(
            nn.Linear(in_features=self.channels, out_features=self.channels//self.r, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=self.channels//self.r, out_features=self.channels, bias=True))

    def forward(self, x):
        max = F.adaptive_max_pool2d(x, output_size=1)
        avg = F.adaptive_avg_pool2d(x, output_size=1)
        b, c, _, _ = x.size()
        linear_max = self.linear(max.view(b,c)).view(b, c, 1, 1)
        linear_avg = self.linear(avg.view(b,c)).view(b, c, 1, 1)
        output = linear_max + linear_avg
        output = F.sigmoid(output) * x
        return output

class SAM(nn.Module):
    def __init__(self, bias=False):
        super(SAM, self).__init__()
        self.bias = bias
        self.conv = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=7, stride=1, padding=3, dilation=1, bias=self.bias)

    def forward(self, x):
        max = torch.max(x,1)[0].unsqueeze(1)
        avg = torch.mean(x,1).unsqueeze(1)
        concat = torch.cat((max,avg), dim=1)
        output = self.conv(concat)
        output = F.sigmoid(output) * x
        return output

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Paper_model(nn.Module):
    def __init__(self):
        super(Paper_model, self).__init__()
        self.layer1 = DeformableConv2d(3, 32, bias=True)
        self.layer2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1, bias=True)
        self.attention1 = CBAM(32, r=2)
        self.bn1 = nn.LayerNorm([32, 52, 52])
        self.pool1 = nn.MaxPool2d(2)

        self.layer3 = DeformableConv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1, bias=True)
        self.layer4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=True)
        self.attention2 = CBAM(64, r=2)
        self.pool2 = nn.MaxPool2d(2)
        self.bn2 = nn.LayerNorm([64, 26, 26])

        self.layer5 = DeformableConv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1, bias=True)
        self.layer6 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=True)
        self.attention3 = CBAM(128, r=2)
        self.pool3 = nn.MaxPool2d(2)
        self.bn3 = nn.LayerNorm([128, 13, 13])

        self.dropout = nn.Dropout(0.5)
        self.pooling = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(128, 8)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.bn1(x)
        x = F.gelu(x)
        x = self.attention1(x)
        x = self.pool1(x)

        x = self.layer3(x)
        x = self.layer4(x)
        x = self.bn2(x)
        x = F.gelu(x)
        x = self.attention2(x)
        x = self.pool2(x)

        x = self.layer5(x)
        x = self.layer6(x)
        x = self.bn3(x)
        x = F.gelu(x)
        x = self.attention3(x)
        x = self.pool3(x)

        x = self.pooling(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return torch.sigmoid(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dcn_model = Paper_model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(dcn_model.parameters(), lr=0.001)
print(dcn_model)

Paper_model(
  (layer1): DeformableConv2d(
    (offset_conv): Conv2d(3, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (modulator_conv): Conv2d(3, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (regular_conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (layer2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (attention1): CBAM(
    (sam): SAM(
      (conv): Conv2d(2, 1, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
    )
    (cam): CAM(
      (linear): Sequential(
        (0): Linear(in_features=32, out_features=16, bias=True)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=16, out_features=32, bias=True)
      )
    )
  )
  (bn1): LayerNorm((32, 52, 52), eps=1e-05, elementwise_affine=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer3): DeformableConv2d(
    (offset_conv): Conv2d(32, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler

class WaferDataset(Dataset):
    def __init__(self, img_array, label_array):
        self.img_array = img_array
        self.label_array = label_array

    def __len__(self):
        return len(self.img_array)

    def __getitem__(self, idx):
        img = torch.from_numpy(self.img_array[idx]).float()
        if img.ndim == 2:  # If the image is grayscale
            img = img.unsqueeze(0).repeat(3, 1, 1)  # Repeat the single channel 3 times
        elif img.ndim == 3:  # If the image is already multi-channel but in [H, W, C] format
            img = img.permute(2, 0, 1)  # Rearrange from [H, W, C] to [C, H, W]
        label = torch.from_numpy(self.label_array[idx]).float()
        return img, label

# 加载数据
data = np.load('/content/drive/MyDrive/DeepLearning_project/Dataset/images_3chnl.npz') # V3_dataset.npz
img_array = data['arr_0']
label_array = data['arr_1']

# 创建数据集
wafer_dataset = WaferDataset(img_array, label_array)

# 设置划分比例
val_split = 0.2
dataset_size = len(wafer_dataset)
indices = list(range(dataset_size))
np.random.shuffle(indices)
val_size = int(np.floor(val_split * dataset_size))
train_indices, val_indices = indices[val_size:], indices[:val_size]

# 创建训练集和验证集的 SubsetRandomSampler
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

# 创建数据加载器
batch_size = 32
train_dataloader = DataLoader(wafer_dataset, batch_size=batch_size, sampler=train_sampler)
val_dataloader = DataLoader(wafer_dataset, batch_size=batch_size, sampler=val_sampler)


In [ ]:
img_array.shape

(38015, 52, 52, 3)

In [ ]:
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

best_acc = 0.9740

# 创建数据集
wafer_dataset = WaferDataset(img_array, label_array)

# 创建数据加载器
batch_size = 32
dataloader = DataLoader(wafer_dataset, batch_size=batch_size, shuffle=True)

# 使用数据加载器进行训练
# 创建模型实例
paper_model = Paper_model()

# 定义损失函数
criterion = nn.BCELoss()

# 定义优化器
optimizer = optim.Adam(paper_model.parameters(), lr=0.001)  # 可以调整学习率


num_epochs = 30
# 检查是否有可用的 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 将模型移动到设备上
paper_model.to(device)

# 加载数据集的代码

# 创建数据加载器，并将数据移动到设备上
dataloader = DataLoader(wafer_dataset, batch_size=batch_size, shuffle=True)
dataloader = [(imgs.to(device), labels.to(device)) for imgs, labels in dataloader]

import random
from tqdm import tqdm
def caculate_acc_num(outputs,labels):
  acc_num = 0
  predicted = (outputs > 0.5).float()
  #print(outputs.shape)
  for idx in range(predicted.shape[0]):
    #print("Predicted : ",predicted[idx])
    #print("Labels : ",labels[idx])
    if torch.allclose(predicted[idx], labels[idx]):
      acc_num += 1

  return acc_num
# 使用数据加载器进行训练
for epoch in range(num_epochs):
    train_acc_num = 0
    total_train = 0
    train_loss = 0
    for imgs, labels in tqdm(train_dataloader):
        # 清除梯度
        optimizer.zero_grad()
        imgs = imgs.to(device)
        labels = labels.to(device)
        # 前向传播
        outputs = paper_model(imgs)
        total_train += imgs.shape[0]
        train_acc_num += caculate_acc_num(outputs,labels)
        # 计算损失
        loss = criterion(outputs, labels)
        train_loss += loss.item()
        # 反向传播
        loss.backward()

        # 更新权重
        optimizer.step()
        # 随机选择一笔数据打印其标签和预测值
        #idx = random.randint(0, len(labels) - 1)
        #print(f'Label: {labels[idx]}, Prediction: {(outputs[idx] > 0.5).float()}')

    val_acc_num = 0
    total_val = 0
    val_loss = 0
    for imgs, labels in tqdm(val_dataloader):
        # 前向传播
        imgs = imgs.to(device)
        labels = labels.to(device)
        outputs = paper_model(imgs)
        total_val += imgs.shape[0]
        val_acc_num += caculate_acc_num(outputs, labels)
        # 计算损失
        loss = criterion(outputs, labels)
        val_loss += loss.item()


    # 每个 epoch 结束后打印损失
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {(train_loss/total_train):.4f} , Acc : {(train_acc_num/total_train):.4f}')
    print(f'Val_loss: {(val_loss/total_val):.4f}, Val_Acc: {(val_acc_num/total_val):.4f}')

    if (val_acc_num/total_val) > best_acc :
      best_acc = (val_acc_num/total_val)
      torch.save(paper_model.state_dict(), f'/content/drive/MyDrive/DeepLearning_project/Pytorch_version_dfc/V7_weight/weight_{best_acc:.4f}.pth')

Using device: cuda


100%|██████████| 238/238 [00:03<00:00, 66.76it/s]


Epoch [1/30], Loss: 0.0050 , Acc : 0.6393
Val_loss: 0.0016, Val_Acc: 0.8891


100%|██████████| 238/238 [00:03<00:00, 76.59it/s]


Epoch [2/30], Loss: 0.0011 , Acc : 0.9370
Val_loss: 0.0009, Val_Acc: 0.9498


...

Epoch [28/30], Loss: 0.0001 , Acc : 0.9940
Val_loss: 0.0006, Val_Acc: 0.9722


100%|██████████| 238/238 [00:03<00:00, 63.75it/s]


Epoch [29/30], Loss: 0.0001 , Acc : 0.9922
Val_loss: 0.0004, Val_Acc: 0.9799


100%|██████████| 238/238 [00:03<00:00, 76.44it/s]

Epoch [30/30], Loss: 0.0001 , Acc : 0.9946
Val_loss: 0.0004, Val_Acc: 0.9797


In [ ]:
best_acc

0.9815862159673813

In [ ]:
paper_model = Paper_model()  # create an instance of the model
paper_model.load_state_dict(torch.load(f'/content/drive/MyDrive/weight_0.9816.pth'))
paper_model.eval()

Paper_model(
  (layer1): DeformableConv2d(
    (offset_conv): Conv2d(3, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (modulator_conv): Conv2d(3, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (regular_conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (layer2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (attention1): CBAM(
    (sam): SAM(
      (conv): Conv2d(2, 1, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
    )
    (cam): CAM(
      (linear): Sequential(
        (0): Linear(in_features=32, out_features=16, bias=True)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=16, out_features=32, bias=True)
      )
    )
  )
  (bn1): LayerNorm((32, 52, 52), eps=1e-05, elementwise_affine=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer3): DeformableConv2d(
    (offset_conv): Conv2d(32, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [ ]:
!pip install thop
from thop import profile  # Import the profiler
import torch.optim as optim
from tqdm import tqdm

# 定义损失函数
criterion = nn.BCELoss()
# 定义优化器
optimizer = optim.Adam(paper_model.parameters(), lr=0.001)  # 可以调整学习率

def calculate_acc_num(outputs, labels):
    acc_num = 0
    predicted = (outputs > 0.5).float()
    for idx in range(predicted.shape[0]):
        if torch.allclose(predicted[idx], labels[idx]):
            acc_num += 1
    return acc_num

def evaluate_model(model, dataloader, device):
    model.to(device)
    model.eval()

    val_acc_num = 0
    total_val = 0
    val_loss = 0

    # Assume the first batch to infer the input size for FLOPs calculation
    first_batch = next(iter(dataloader))
    imgs, _ = first_batch
    input_shape = imgs.shape  # Assuming imgs_ori to be representative

    # Calculate FLOPs and Parameters
    flops, params = profile(model, inputs=(imgs.to(device), ), verbose=False)

    # Convert FLOPs to GigaFLOPs and parameters to thousands (K)
    flops_in_gflops = flops / 1e9  # Convert from FLOPs to GFLOPs
    params_in_k = params / 1e3     # Convert from parameters to thousands

    # Print the results formatted as GFLOPs and K
    print(f"FLOPs: {flops_in_gflops:.4f} GFLOPs   Params: {params_in_k:.3f}K")

    for imgs, labels in tqdm(dataloader):
        imgs = imgs.to(device)
        labels = labels.to(device)
        outputs = model(imgs)
        total_val += imgs.shape[0]
        val_acc_num += calculate_acc_num(outputs, labels)
        loss = criterion(outputs, labels)
        val_loss += loss.item()

    print(f'Val_loss: {val_loss / total_val:.4f}, Val_Acc: {val_acc_num / total_val:.4f}')
    return outputs

outputs = evaluate_model(paper_model, val_dataloader, torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

100%|██████████| 238/238 [00:03<00:00, 64.47it/s]

Val_loss: 0.0004, Val_Acc: 0.9816


In [ ]:
CLASS_MAPPING = {
    "[0 0 0 0 0 0 0 0]":0,
    "[1 0 0 0 0 0 0 0]":1,
    "[0 1 0 0 0 0 0 0]":2,
    "[0 0 1 0 0 0 0 0]":3,
    "[0 0 0 1 0 0 0 0]":4,
    "[0 0 0 0 1 0 0 0]":5,
    "[0 0 0 0 0 1 0 0]":6,
    "[0 0 0 0 0 0 1 0]":7,
    "[0 0 0 0 0 0 0 1]":8,
    "[1 0 1 0 0 0 0 0]":9,
    "[1 0 0 1 0 0 0 0]":10,
    "[1 0 0 0 1 0 0 0]":11,
    "[1 0 0 0 0 0 1 0]":12,
    "[0 1 1 0 0 0 0 0]":13,
    "[0 1 0 1 0 0 0 0]":14,
    "[0 1 0 0 1 0 0 0]":15,
    "[0 1 0 0 0 0 1 0]":16,
    "[0 0 1 0 1 0 0 0]":17,
    "[0 0 1 0 0 0 1 0]":18,
    "[0 0 0 1 1 0 0 0]":19,
    "[0 0 0 1 0 0 1 0]":20,
    "[0 0 0 0 1 0 1 0]":21,
    "[1 0 1 0 1 0 0 0]":22,
    "[1 0 1 0 0 0 1 0]":23,
    "[1 0 0 1 1 0 0 0]":24,
    "[1 0 0 1 0 0 1 0]":25,
    "[1 0 0 0 1 0 1 0]":26,
    "[0 1 1 0 1 0 0 0]":27,
    "[0 1 1 0 0 0 1 0]":28,
    "[0 1 0 1 1 0 0 0]":29,
    "[0 1 0 1 0 0 1 0]":30,
    "[0 1 0 0 1 0 1 0]":31,
    "[0 0 1 0 1 0 1 0]":32,
    "[0 0 0 1 1 0 1 0]":33,
    "[1 0 1 0 1 0 1 0]":34,
    "[1 0 0 1 1 0 1 0]":35,
    "[0 1 1 0 1 0 1 0]":36,
    "[0 1 0 1 1 0 1 0]":37
}

#%%
key = "[0 1 0 1 1 0 1 0]"
value = CLASS_MAPPING["[0 1 0 1 1 0 1 0]"]
print(value)
#%%
import torch
from tqdm import tqdm
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, recall_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def plot_confusion_matrix_and_accuracy(model, dataloader, device, class_mapping):
    num_classes = len(class_mapping)
    class_labels = [None] * num_classes
    for label_str, index in class_mapping.items():
        class_labels[index] = str(index)  # Directly use the numerical identifier

    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for imgs, labels in tqdm(dataloader):
            imgs = imgs.to(device)
            labels = labels.to(device)
            outputs = model(imgs)
            predicted = (outputs > 0.5).float()

            for i in range(labels.size(0)):
                label_vec = labels[i].to(torch.uint8).tolist()
                label_str = '[' + ' '.join(map(str, label_vec)) + ']'
                class_idx = class_mapping[label_str]

                pred_vec = predicted[i].tolist()
                pred_str = '[' + ' '.join(str(int(p)) for p in pred_vec) + ']'
                pred_idx = class_mapping.get(pred_str, -1)  # Handle unseen/misformatted predictions

                all_labels.append(class_idx)
                all_predictions.append(pred_idx)

    # Compute the confusion matrix
    conf_mat = confusion_matrix(all_labels, all_predictions, labels=range(len(class_mapping)))

    # # Plotting the confusion matrix
    # plt.figure(figsize=(15, 13))  # Increase figure size
    # ax = sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    # plt.title('Confusion Matrix', size=20)  # Increase title font size
    # plt.ylabel('True Label', size=18)  # Increase y-axis label font size
    # plt.xlabel('Predicted Label', size=18)  # Increase x-axis label font size

    # # Improve layout to prevent cut-off issues
    # plt.xticks(rotation=90, size=10)  # Rotate x labels for better fit, adjust size as needed
    # plt.yticks(size=10)  # Adjust y labels size as needed
    # plt.tight_layout()  # This adjusts subplot params so that the subplot(s) fits in to the figure area

    # plt.show()

    # Printing classification report
    print(classification_report(
        all_labels,
        all_predictions,
        target_names=class_labels,
        labels=range(len(class_labels))  # Ensure it considers all classes
    ))

    # Calculate and print class-specific accuracy
    class_accuracy = 100 * conf_mat.diagonal() / conf_mat.sum(axis=1)
    for i, accuracy in enumerate(class_accuracy):
        if not np.isnan(accuracy):
            print(f'Accuracy of class {class_labels[i]} : {accuracy:.2f}%')
        else:
            print(f'No samples for class {class_labels[i]}')

# Call the function
plot_confusion_matrix_and_accuracy(paper_model, val_dataloader, device, CLASS_MAPPING)

37


100%|██████████| 238/238 [00:02<00:00, 94.58it/s]


              precision    recall  f1-score   support

           0       0.99      1.00      0.99       194
           1       0.98      0.99      0.98       222
           2       0.97      1.00      0.99       210
           3       0.99      0.98      0.99       180
           4       0.98      0.99      0.99       248
           5       0.99      0.98      0.98       193
           6       1.00      1.00      1.00        24
           7       0.98      0.97      0.97       174
           8       1.00      0.96      0.98       175
           9       0.99      0.98      0.98       214
          10       0.98      1.00      0.99       203
          11       0.97      0.98      0.98       189
          12       0.99      0.99      0.99       202
          13       0.99      0.97      0.98       215
          14       0.97      1.00      0.99       211
          15       0.98      0.97      0.97       203
          16       0.98      1.00      0.99       212
          17       0.96    